In [ ]:
%matplotlib inline
import tables
import numpy
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib

import glob
import sys
import matplotlib.gridspec as gridspec
import CalculateMeanWinds
from scipy import signal


In [ ]:
def readafile(fname):
    '''
    Function written by nicolls to read
    '''
    h5file=tables.open_file(fname)
    output={}
    for group in h5file.walk_groups("/"):
        output[group._v_pathname]={}
        for array in h5file.list_nodes(group, classname = 'Array'):
            output[group._v_pathname][array.name]=array.read()
    h5file.close()

    return output

def WeightedMeanNaN(x, dx):
    xdx = x/dx
    # bevington 4.17
    weightedMean = numpy.nansum(x/(dx**2), axis=0)/numpy.nansum(1./dx**2, axis=0)
    # bevington 4.19
    weightedVar = 1./numpy.nansum((1./dx**2), axis=0)
    weightedStd = numpy.sqrt(weightedVar)
    n = [numpy.where(numpy.isfinite(xdx[:,i]))[0].shape[0] for i in range(xdx.shape[1])]
    # print n
    return weightedMean, weightedStd, n

In [ ]:
def PlotWindsvFlow(inDict,KeySelect, outfile):
    nrow = 7
    ncol = 3
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(12,12))
    gs = gridspec.GridSpec(nrow, ncol)
    ialt = 8 # sets the altitude we are interested in
    iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    print keyList
    
    ikey = 0
    irow = 0
    
    nshape = inDict['201206']['ut']['DecimalHoursTimeGrid'].shape[0]
    
    colors = plt.cm.brg(numpy.linspace(0,1,len(keyList)))
    for irow in range(1,7,1):
        AvMeridWind = numpy.zeros([nshape,len(keyList)])
        AvZonalWind = numpy.zeros([nshape,len(keyList)])
        AvZonalFlow = numpy.zeros([nshape,len(keyList)])
        AvMeridFlow = numpy.zeros([nshape,len(keyList)])
        for ikey in range(len(keyList)):
            mltTimeDict = inDict[keyList[ikey]][KeySelect]
            ax00 = plt.subplot(gs[0,0])
            ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalFlow'][1,:], color=colors[ikey])
            AvZonalFlow[:,ikey] = mltTimeDict['MeanZonalFlow'][1,:]
            ax00.set_ylim([-400,400])
            ax00.set_yticks([-200,0,200])
            ax00.set_xticks(numpy.arange(0,26,2))
            ax00.set_xlim([0,24])
#             ax00.xaxis.set_ticklabels([])
            ax00.grid()
            ax00.set_title('Zonal')

            ax01 = plt.subplot(gs[0,1])
            ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], color=colors[ikey])
    
            AvMeridFlow[:,ikey] = mltTimeDict['MeanMeridFlow'][1,:]
            ax01.set_ylim([-400,400])
            ax01.set_yticks([-200,0,200])
            ax01.set_xticks(numpy.arange(0,26,2))
            ax01.set_xlim([0,24])
#             ax01.xaxis.set_ticklabels([])
            ax01.grid()
            ax01.set_title('Meridional')

            # now do the winds
            ax1 = plt.subplot(gs[irow,0]) # plot winds
            ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalWinds'][0,:,ialt], color=colors[ikey])
            ax1.set_ylabel('%0.1f'%mltTimeDict['MeanAltitude'][ialt])
            ax1.set_ylim([-200,200])
            AvZonalWind[:,ikey] = mltTimeDict['MeanZonalWinds'][0,:,ialt]
            ax1.set_xticks(numpy.arange(0,26,2))
            ax1.set_xlim([0,24])
#             ax1.xaxis.set_ticklabels([])
            ax1.grid()

            ax2 = plt.subplot(gs[irow,1])
            ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridWinds'][0,:,ialt], color=colors[ikey])
            AvMeridWind[:,ikey] = mltTimeDict['MeanMeridWinds'][0,:,ialt]
            ax2.set_ylim([-200,200])
            ax2.set_xticks(numpy.arange(0,26,2))
            ax2.set_xlim([0,24])
#             ax2.xaxis.set_ticklabels([])
            ax2.grid()
#             print 'ikey', ikey
            ikey=ikey+1
        
        ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalFlow, axis=1), 'k-', lw=2)
        ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridFlow, axis=1), 'k-', lw=2)
        ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalWind,axis=1), 'k-', lw=2)
        ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridWind,axis=1), 'k-', lw=2)
        
        periods = numpy.arange(1.,48.,0.1)
        frequencies = (2.*numpy.pi)/periods
        
        LS_ZFlow = signal.lombscargle(mltTimeDict['DecimalHoursTimeGrid'],\
                                      numpy.nanmean(AvZonalFlow, axis=1),frequencies,\
                                     normalize=True)
        LS_MFlow = signal.lombscargle(mltTimeDict['DecimalHoursTimeGrid'],\
                                      numpy.nanmean(AvMeridFlow, axis=1),frequencies,\
                                     normalize=True)
        
        ax03 = plt.subplot(gs[0,2])
        ax03twin = ax03.twinx()
        ax03.plot(periods,LS_ZFlow, 'b-')
        ax03.tick_params('y', colors='b')
        ax03twin.plot(periods,LS_MFlow, 'r-')
        ax03twin.tick_params('y', colors='r')
        ax03.set_xlim([0,25])
        ax03.set_ylim([0,1])
        ax03twin.set_ylim([0,1])
        ax03.plot([4,4], [0,1], 'k--')
        ax03.plot([8,8], [0,1], 'k--')
        ax03.plot([12,12], [0,1], 'k--')
        ax03.plot([24,24], [0,1], 'k--')
        ax03.set_ylabel('Meridional')
        ax03twin.set_ylabel('Zonal')
        ax03.set_xticks(numpy.arange(0,26,2))
        ax03.grid()
      
        
        
        LS_ZWind = signal.lombscargle(mltTimeDict['DecimalHoursTimeGrid'],\
                                      numpy.nanmean(AvZonalWind, axis=1),frequencies,\
                                     normalize=True)
        LS_MWind = signal.lombscargle(mltTimeDict['DecimalHoursTimeGrid'],\
                                      numpy.nanmean(AvMeridWind, axis=1),frequencies,\
                                     normalize=True)
        
        ax3 = plt.subplot(gs[irow,2])
        ax3twin = ax3.twinx()
        ax3.plot(periods,LS_ZWind, 'b-', label='Z')
        ax3.tick_params('y', colors='b')
        ax3twin.plot(periods,LS_MWind, 'r-', label='M')
        ax3twin.tick_params('y', colors='r')
        ax3.set_xlim([0,25])
        ax3.set_ylim([0,1])
        ax3twin.set_ylim([0,1])
        ax3.plot([4,4], [0,1], 'k--')
        ax3.plot([8,8], [0,1], 'k--')
        ax3.plot([12,12], [0,1], 'k--')
        ax3.plot([24,24], [0,1], 'k--')
        ax3.set_ylabel('Meridional')
        ax3twin.set_ylabel('Zonal')
        ax3.set_xticks(numpy.arange(0,26,2))
        ax3.grid()
        ax3.xaxis.set_ticklabels([])
        
        
#         ax.set_ylabel('Zonal - U (m/s)', color='b')
#         ax2.set_ylabel('Meridional - V (m/s)', color='r')
#         ax.set_title('%s at %s km'%(keyList[ikey],mltTimeDict['MeanAltitude'][iplot1]))
#         ax.tick_params('y', colors='b')
#         ax2.tick_params('y', colors='r')
        
        ialt=ialt-1
    
    
        
        
    
#     for icol in range(0,ncol,1):
# #         print ikey, keyList[ikey]
        
#         for irow in range(0,nrow,1):
#             mltTimeDict = inDict[keyList[ikey]][KeySelect]
#             print 'irow,icol',totalNumPlots,irow,icol, keyList[ikey]
#             ax = plt.subplot(gs[irow,icol])
#             ax2 = ax.twinx()
#             ax.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalFlow'][1,:], 'b-', label='Zonal -U')
#             ax.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalWinds'][0,:,iplot1], 'bx')
#             ax.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalWinds'][0,:,iplot1], 'b--')
#             ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], 'r-', label='Meridional - V')
#             ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridWinds'][0,:,iplot1], 'rx')
#             ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridWinds'][0,:,iplot1], 'r--')
# #             ax.set_ylim([-250,250])
# #             ax2.set_ylim([-250,250])
#             ax.set_ylabel('Zonal - U (m/s)', color='b')
#             ax2.set_ylabel('Meridional - V (m/s)', color='r')
#             ax.set_title('%s at %s km'%(keyList[ikey],mltTimeDict['MeanAltitude'][iplot1]))
#             ax.tick_params('y', colors='b')
#             ax2.tick_params('y', colors='r')
#             ax.grid()
        
                             
            
        
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    plt.tight_layout()
    plt.savefig(outfile)

In [ ]:
def PlotWindsvFlowVAltitude(inDict,KeySelect, outfile):
    """
    This function is primarily to plot the mean winds for each altitude as a function of year.
    This accepts a single dictionary in
    
    Taking out the 92.5 km altitude bin
    """
    nrow = 8
    ncol = 2
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(6,8))
    gs = gridspec.GridSpec(nrow, ncol)
    ialt = 8 # sets the altitude we are interested in
    iplot2 = 8
    iloc = 1
    
    keyList = sorted(inDict.keys())
    print keyList
    
    ikey = 0
    irow = 0
    
    tmpKey = inDict.keys()
    nshape = inDict[tmpKey[0]]['ut']['DecimalHoursTimeGrid'].shape[0]
    
#     colors = plt.cm.gist_rainbow(numpy.linspace(0,1,len(keyList)))
    colors = plt.cm.brg(numpy.linspace(0,1,len(keyList)))
    for irow in range(1,8,1):
        AvMeridWind = numpy.zeros([nshape,len(keyList)])
        AvZonalWind = numpy.zeros([nshape,len(keyList)])
        AvZonalFlow = numpy.zeros([nshape,len(keyList)])
        AvMeridFlow = numpy.zeros([nshape,len(keyList)])
        for ikey in range(len(keyList)):
            mltTimeDict = inDict[keyList[ikey]][KeySelect]
            ax00 = plt.subplot(gs[0,0])
            ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalFlow'][1,:], color=colors[ikey])
            AvZonalFlow[:,ikey] = mltTimeDict['MeanZonalFlow'][1,:]
            ax00.set_ylim([-500,500])
            ax00.set_yticks(numpy.arange(-250,500,250))
            ax00.set_xticks(numpy.arange(0,26,3))
            ax00.set_xlim([0,24])
            ax00.xaxis.set_ticklabels([])
            ax00.grid()
            ax00.set_title('Zonal')
            ax00.set_ylabel('Ions (m/s)', fontsize=10)
#             ax00.set_xlabel(None)

            ax01 = plt.subplot(gs[0,1])
            if irow == 1:
                ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], color=colors[ikey],label='%s'%keyList[ikey][0:4])
            else:
                ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], color=colors[ikey])
            AvMeridFlow[:,ikey] = mltTimeDict['MeanMeridFlow'][1,:]
            ax01.set_ylim([-500,500])
            ax01.set_yticks(numpy.arange(-250,500,250))
            ax01.set_xticks(numpy.arange(0,26,3))
            ax01.set_xlim([0,24])
            ax01.xaxis.set_ticklabels([])
            ax01.grid()
            ax01.set_title('Meridional')
            ax01.yaxis.set_ticklabels([])
#             ax01.set_xlabel(None)
            

            # now do the winds
            ax1 = plt.subplot(gs[irow,0]) # plot winds
            ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalWinds'][0,:,ialt], color=colors[ikey])
            ax1.set_ylabel(r'%0.1f km'%mltTimeDict['MeanAltitude'][ialt]+'\n'+'(m/s)', fontsize=10)
            ax1.set_ylim([-200,200])
            ax1.set_yticks([-100,0,100])
            AvZonalWind[:,ikey] = mltTimeDict['MeanZonalWinds'][0,:,ialt]
            ax1.set_xticks(numpy.arange(0,26,3))
            timeLbl = ['%02d'%i for i in numpy.arange(0,26,3)]
            ax1.set_xticklabels(timeLbl)
            ax1.set_xlim([0,24])
            if irow != 7:
                ax1.xaxis.set_ticklabels([])
            else:
                ax1.set_xlabel('MLT (Dec. Hrs)')
            ax1.grid()
#             ax1.set_xlabel(None)
        

            ax2 = plt.subplot(gs[irow,1])
            ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridWinds'][0,:,ialt], color=colors[ikey])
            AvMeridWind[:,ikey] = mltTimeDict['MeanMeridWinds'][0,:,ialt]
            ax2.set_yticks([-100,0,100])
            ax2.set_ylim([-200,200])
            ax2.set_xticks(numpy.arange(0,26,3))
            timeLbl = ['%02d'%i for i in numpy.arange(0,26,3)]
            ax2.set_xticklabels(timeLbl)
            ax2.set_xlim([0,24])
            if irow != 7:
                ax2.xaxis.set_ticklabels([])
            else:
                ax2.set_xlabel('MLT (Dec. Hrs)')
#             ax2.xaxis.set_ticklabels([])
            
            ax2.yaxis.set_ticklabels([])
            ax2.grid()
#             ax2.set_xlabel(None)
#             print 'ikey', ikey
            ikey=ikey+1
        
        ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalFlow, axis=1), 'k-', lw=2)
        ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridFlow, axis=1), 'k-', lw=2)
        ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalWind,axis=1), 'k-', lw=2)
        ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridWind,axis=1), 'k-', lw=2)
        
        ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax1.plot(mltTimeDict['DecimalHoursTimeGrid'],numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        
        
#         ax00.grid()
#         ax01.grid()
#         ax1.grid()
#         ax2.grid()


        
        ialt=ialt-1
    
    
        
        
    

            
        
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    handles, labels = ax01.get_legend_handles_labels()
#     fig.legend(handles, labels, loc='upper center')
    ax01.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()
    plt.savefig(outfile)

In [ ]:
def PlotMultipleDictionariesVsAltitude(DictList,KeySelect,LabelList,FileName=None):
    """
    The purpose of this function is to plot different configurations as a function of altitude.
    In this case, I will probably look at each year individual to compare different filtering with the overall results.
    """
    nrow = 7
    ncol = 2
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    
    # will want to loop for each year, for each altitude.
    # new figure for each year.
    
    # get the keys from the first dictionary
    keyList = sorted(DictList[0].keys())
    
    # key list is the year-month
    for ikey in keyList:
        f = plt.figure(dpi=300,figsize=(8.5,11))
        gs = gridspec.GridSpec(nrow, ncol)
        ialt = 8 # sets the altitude we are interested in
        iplot2 = 8
        iloc = 1
        icolor = 0
        idict = 0
        
        for inDict in DictList:
        
            # get the shape, colors and data input
            nshape = inDict[ikey]['ut']['DecimalHoursTimeGrid'].shape[0]
#             colors = plt.cm.gist_rainbow(numpy.linspace(0,1,len(DictList)))
            colors = ['r', 'b','g','m','o']
            mltTimeDict = inDict[ikey][KeySelect]
            print 'ikey, KeySelect', ikey,KeySelect
            print mltTimeDict.keys()
            
            # loop with respect to altitude now
            
            ax00 = plt.subplot(gs[0,0])
            ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalFlow'][1,:],\
                      color=colors[icolor], label=LabelList[idict])
#                     AvZonalFlow[:,ikey] = mltTimeDict['MeanZonalFlow'][1,:]
            ax00.set_ylim([-500,500])
            ax00.set_yticks(numpy.arange(-250,500,250))
            ax00.set_xticks(numpy.arange(0,26,3))
            ax00.set_xlim([0,24])
            ax00.xaxis.set_ticklabels([])
            ax00.grid()
            ax00.set_title('Zonal')
            ax00.set_ylabel('Ions (m/s)', fontsize=10)
#             ax00.set_xlabel(None)

            ax01 = plt.subplot(gs[0,1])
            ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], \
                      color=colors[icolor], label=LabelList[idict])
#                     AvMeridFlow[:,ikey] = mltTimeDict['MeanMeridFlow'][1,:]
            ax01.set_ylim([-500,500])
            ax01.set_yticks(numpy.arange(-250,500,250))
            ax01.set_xticks(numpy.arange(0,26,3))
            ax01.set_xlim([0,24])
            ax01.xaxis.set_ticklabels([])
            ax01.grid()
            ax01.set_title('Meridional')
            ax01.yaxis.set_ticklabels([])
    #             ax01.set_xlabel(None)
            
            
            ialt=8
            for irow in range(1,7,1):
#                 AvMeridWind = numpy.zeros([nshape,len(keyList)])
#                 AvZonalWind = numpy.zeros([nshape,len(keyList)])
#                 AvZonalFlow = numpy.zeros([nshape,len(keyList)])
#                 AvMeridFlow = numpy.zeros([nshape,len(keyList)])
                
#                 if irow == 1:
#                     ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], color=colors[icolor])
#                 else:
#                     ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridFlow'][1,:], color=colors[icolor])
                


                # now do the winds
                ax1 = plt.subplot(gs[irow,0]) # plot winds
                ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanZonalWinds'][0,:,ialt], color=colors[icolor])
                ax1.set_ylabel(r'%0.1f km'%mltTimeDict['MeanAltitude'][ialt]+'\n'+'(m/s)', fontsize=10)
                ax1.set_ylim([-200,200])
                ax1.set_yticks([-100,0,100])
#                     AvZonalWind[:,ikey] = mltTimeDict['MeanZonalWinds'][0,:,ialt]
                ax1.set_xticks(numpy.arange(0,26,3))
                timeLbl = ['%02d'%i for i in numpy.arange(0,26,3)]
                ax1.set_xticklabels(timeLbl)
                ax1.set_xlim([0,24])
                if irow != 6:
                    ax1.xaxis.set_ticklabels([])
                else:
                    ax1.set_xlabel('MLT (Dec. Hrs)')
                ax1.grid()
    #             ax1.set_xlabel(None)


                ax2 = plt.subplot(gs[irow,1])
                ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], mltTimeDict['MeanMeridWinds'][0,:,ialt], color=colors[icolor])
#                     AvMeridWind[:,ikey] = mltTimeDict['MeanMeridWinds'][0,:,ialt]
                ax2.set_yticks([-100,0,100])
                ax2.set_ylim([-200,200])
                ax2.set_xticks(numpy.arange(0,26,3))
                timeLbl = ['%02d'%i for i in numpy.arange(0,26,3)]
                ax2.set_xticklabels(timeLbl)
                ax2.set_xlim([0,24])
                if irow != 7:
                    ax2.xaxis.set_ticklabels([])
                else:
                    ax2.set_xlabel('MLT (Dec. Hrs)')
    #             ax2.xaxis.set_ticklabels([])
                
                ax2.yaxis.set_ticklabels([])
    #             ax2.set_xlabel(None)
    #             print 'ikey', ikey
                
                ialt=ialt-1
                ax00.grid()
                ax01.grid()
                ax1.grid()
                ax2.grid()
                ax1.plot(mltTimeDict['DecimalHoursTimeGrid'],numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
                ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
            icolor=icolor+1
            idict=idict+1

#                 ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalFlow, axis=1), 'k-', lw=2)
#                 ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridFlow, axis=1), 'k-', lw=2)
#                 ax1.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvZonalWind,axis=1), 'k-', lw=2)
#                 ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.nanmean(AvMeridWind,axis=1), 'k-', lw=2)

        ax00.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax01.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax1.plot(mltTimeDict['DecimalHoursTimeGrid'],numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)
        ax2.plot(mltTimeDict['DecimalHoursTimeGrid'], numpy.zeros(mltTimeDict['DecimalHoursTimeGrid'].shape[0]), 'k--', lw=2)


        


                






            #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                                # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                                # if (irow == nrow-1) and (icol == ncol-1):
                                # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                                # iplot+=1
            #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
            #             ax.set_title(title, fontsize=10)
        handles, labels = ax01.get_legend_handles_labels()
        #     fig.legend(handles, labels, loc='upper center')
        ax01.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
        plt.tight_layout()
        if len(FileName) != 0:
            outfile = './Figures/%s_Comparison_%s.png'%(ikey,FileName)
        else:
            outfile = './Figures/%s_Comparison.png'%ikey
        plt.savefig(outfile)
        plt.close()

# Making the Main Figure

In [ ]:
# import in the new dictionary calculator
# this is the test config file
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofileTEST.ini'

# this is the config file for the paper - 02212019
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/ConfigFileForPaper_02212019.ini'
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)

In [ ]:
# fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200806_Winds.v0.4.2018.03.15.test.h5',
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200906_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201006_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201106_Winds.v0.4.2018.03.15.test.h5',
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201206_Winds.v0.4.2018.03.15.test.h5',
#           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201306_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201406_Winds.v0.4.2018.03.15.test.h5',
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201506_Winds.v0.4.2018.03.15.test.h5']




# pulled the 2017 data

fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201006_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201106_Winds.v0.4.5.2018.11.20.h5',
           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201206_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201306_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201406_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201506_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201606_Winds.v0.4.5.2018.11.20.h5',
         ]

# '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201706_Winds.v0.4.5.2018.11.20.h5'
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.6.2019.01.25/MonthlyWinds.01252018/201706_Winds.v0.4.6.2019.01.25.h5'

# TmpDict = MakeDictionaryList(fnames)
TmpDict = CalcMeanWinds.main(fnames)
PlotWindsvFlowVAltitude(TmpDict,'mlt', './FiguresForPaper/MeanWindsVYear.pdf')

In [ ]:
PlotWindsvFlowVAltitude(TmpDict,'mlt', './Figures/AltvUTTimeTEST.png')

In [ ]:
# now with new data in

fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201006_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201106_Winds.v0.4.5.2018.11.20.h5',
           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201206_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201306_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201406_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201506_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201606_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.6.2019.01.25/MonthlyWinds.01252018/201706_Winds.v0.4.6.2019.01.25.h5'
         ]

# '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201706_Winds.v0.4.5.2018.11.20.h5'
#          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.6.2019.01.25/MonthlyWinds.01252018/201706_Winds.v0.4.6.2019.01.25.h5'

# TmpDict = MakeDictionaryList(fnames)
TmpDict = CalcMeanWinds.main(fnames)
PlotWindsvFlowVAltitude(TmpDict,'mlt', 'AltvUTTime01282019_WithNewData_8e10.png')

In [ ]:
TmpDict['201106']['ut']['MeanAltitude']

In [ ]:
# import in the new dictionary calculator
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofile.ini'
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200806_Winds.v0.4.2018.03.15.test.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/200906_Winds.v0.4.2018.03.15.test.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201006_Winds.v0.4.2018.03.15.test.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201106_Winds.v0.4.2018.03.15.test.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201206_Winds.v0.4.2018.03.15.test.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201306_Winds.v0.4.2018.03.15.test.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201406_Winds.v0.4.2018.03.15.test.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/MonthlyWinds/201506_Winds.v0.4.2018.03.15.test.h5']
# TmpDict = MakeDictionaryList(fnames)
TmpDict = CalcMeanWinds.main(fnames)
PlotWindsvFlowVAltitude(TmpDict,'mlt', 'AltvUTTime.10042018.png')

In [ ]:
fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201706_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.6.2019.01.25/MonthlyWinds.01252018/201706_Winds.v0.4.6.2019.01.25.h5'
         ]

# '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201706_Winds.v0.4.5.2018.11.20.h5'
configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofileTEST.ini'
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
# TmpDict = MakeDictionaryList(fnames)
TmpDict1 = CalcMeanWinds.main(fnames)

CalcMeanWinds2 = CalculateMeanWinds.CalculateMeanWinds('/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofile.ini')
TmpDict2 = CalcMeanWinds2.main(fnames)


# PlotWindsvFlowVAltitude(TmpDict,'mlt', 'AltvUTTime_2017only_01272018_8e10.png')

In [ ]:
DictList = [TmpDict1,TmpDict2]
PlotMultipleDictionariesVsAltitude(DictList,'mlt',['C1','C2'],FileName='TEST')